In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

adapt_dataset = load_dataset("csv", data_files="/content/drive/MyDrive/ru_adapt.csv")
adapt_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1b1c1941c31e4e71/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'source', 'cos_sim', 'target'],
        num_rows: 15166
    })
})

In [ ]:
adapt_df = adapt_dataset['train'].to_pandas()
adapt_df

,Unnamed: 0,source,cos_sim,target
0,1,"— Ах, только не мужем, с простою усмешкой сказ...",0.863004,"— Ах, только не мужем, — с простою усмешкой ск..."
1,2,"Мне скоро шестьдесят, я старик, одинокий, ничт...",0.653318,"Ничего во мне нет хорошего, кроме этой любви к..."
2,4,"Я вполне уверен, что вы раскаялись и раскаивае...",0.621839,"Я уверен, что вы раскаялись и будете содейство..."
3,7,И оба пошли быстро.,0.803202,И они пошли быстро.
4,8,Большинство пожилых людей и люди высокопоставл...,0.874004,Большинство немолодых людей и людей высокопост...
...,...,...,...,...
15161,24226,— Ага!,0.455928,— Ага!
15162,24227,У нас в феврале масленица,0.960889,У нас в феврале масленица.
15163,24228,— Разводиться мне или нет?,0.962737,— Разводиться мне или нет?
15164,24229,"Парусиновую куртку Петр Алексеевич сбросил, ру...",0.938603,"Парусиновую куртку Пётр Алексеевич сбросил, р..."


In [ ]:
source_lines = adapt_df['source'].tolist()
target_lines = adapt_df['target'].tolist()

In [ ]:
source = [i.strip() for i in source_lines]
target = [i.strip() for i in target_lines]

lines = list(zip(source, target))

In [ ]:
def get_x_y(sentence):
    return sentence[0], sentence[1]

In [ ]:
pip install sentence-transformers

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import logging

backbone_model = 'cointegrated/rut5-base-paraphraser'
model = T5ForConditionalGeneration.from_pretrained(backbone_model)
tokenizer = T5Tokenizer.from_pretrained(backbone_model)
model.cuda();
optimizer = torch.optim.Adam(params=[p for p in model.parameters() if p.requires_grad], lr=1e-5)

In [ ]:
import random
from tqdm.auto import tqdm, trange

import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
device = torch.device('cuda')

random.shuffle(lines)

model.train();
batch_size = 10
max_len = 1024
epochs = 10
accumulation_steps = 32
save_steps = 50000
total_samples = 15166  # длина датасета

window = 5000
ewm = 0

tq = trange(int(total_samples / batch_size))

for epoch in range(epochs):
  print('EPOCH', epoch)
  for i in tq:
    xx = []
    yy = []
    for _ in range(batch_size):
      xxx, yyy = get_x_y(random.choice(lines))
      xx.append(xxx)
      yy.append(yyy)

    # кодируем source и target
    try:
      x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
      y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
      # -100 - специальное значение, позволяющее не учитывать токены
      y.input_ids[y.input_ids==0] = -100

      # вычисляем функцию потерь
      loss = model(
          input_ids=x.input_ids,
          attention_mask=x.attention_mask,
          labels=y.input_ids,
          decoder_attention_mask=y.attention_mask,
          return_dict=True
      ).loss
      # делаем шаг градиентного спуска
      loss.backward()
      # print('ok')
    except RuntimeError:
      print(xx)
      loss = None
      cleanup()
      continue

    w = 1 / min(i+1, window)
    ewm = ewm * (1-w) + loss.item() * w
    tq.set_description(f'loss: {ewm}')
    
    if i % accumulation_steps == 0:
      optimizer.step()
      optimizer.zero_grad()
      cleanup()
    
    if i % window == 0 and i > 0:
      print(ewm)
      cleanup()
      # optimizer.param_groups[0]['lr'] *= 0.999
    if i % save_steps == 0 and i > 0:
      MODEL_NAME = f'new_rut5{i}'
      model.save_pretrained(save_directory = MODEL_NAME)
      tokenizer.save_pretrained(save_directory = MODEL_NAME)
      logging.warning(f'saving... {i}')

      try:
        optimizer.step()
        optimizer.zero_grad()
        # eval_losses()
        optimizer.step()
        optimizer.zero_grad()
      except RuntimeError:
        cleanup()

  0%|          | 0/1516 [00:00<?, ?it/s]

EPOCH 0
EPOCH 1
EPOCH 2
EPOCH 3
EPOCH 4
EPOCH 5
EPOCH 6
EPOCH 7
EPOCH 8
EPOCH 9


In [ ]:
model.save_pretrained('new_rut5')
tokenizer.save_pretrained('new_rut5')

('new_rut5/tokenizer_config.json',
 'new_rut5/special_tokens_map.json',
 'new_rut5/spiece.model',
 'new_rut5/added_tokens.json')